This .ipynb file is going to highlight some topics that is interesting for out project highliting what we learned in class and how we linked the knowledge with the small classification task we have in front of us.

The challenges of the project are the same that we find in the NLP domain in general which are the Productivity, ambiguitty, variability, diversity and sparsity of language.


# STEP 2-TOKENIZATION

The diversity of the different languages makes the choice of the tokeniser a bit more difficult.We need a multilingual tokeniser that can deal with different languages and texts . The choice was to use BERT Multilingual (mBERT).

In [3]:
!pip install datasets
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset

# Charger le modèle et le tokenizer mBERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenisation d'un exemple de texte
example_text = "Ceci est un exemple de texte."
tokens = tokenizer.tokenize(example_text)
print(tokens)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['Ceci', 'est', 'un', 'exemple', 'de', 'texte', '.']


Some texts surpasses the number of 512 as we saw in the part 1-Exploratory Data Analysis .512 is the maximum number that the BERT multilingual model can work with.However we assumed that to guess a language 512 "characters" will be enough for our model to guess the language.This is why we decided to truncate all the texts/lines to 512 tokens each.For really small texts we decided to use paddings to also bring them to 512 tokens.This is what we call:text normalisation.

In [5]:
from transformers import BertTokenizer
from datasets import load_dataset

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Charger le fichier CSV avec `datasets`
dataset = load_dataset('csv', data_files={'train': 'train_cleaned_numeric_labels.csv'}, delimiter=',')

# Fonction de tokenisation avec tronquage et padding à une longueur maximale de 512 tokens
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True, max_length=512)

# Appliquer la tokenisation uniquement à la colonne 'Text' du dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Vérifier les premières entrées après tokenisation

print(tokenized_datasets['train'][0])



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/190099 [00:00<?, ? examples/s]

{'Usage': 'Public', 'Text': 'َ قَالَ النَّبِيُّ ص إِنِّي أَتَعَجَّبُ مِمَّنْ يَضْرِبُ امْرَأَتَهُ وَ هُوَ بِالضَّرْبِ أَوْلَى مِنْهَا لَا تَضْرِبُوا نِسَاءَكُمْ بِالْخَشَبِ فَإِنَّ فِيهِ الْقِصَاصَ وَ لَكِنِ اضْرِبُوهُنَّ بِالْجُوعِ وَ الْعُرْيِ حَتَّى تُرِيحُوا [تَرْبَحُوا] فِي الدُّنْيَا وَ الْآخِرَةِ وَ أَيُّمَا رَجُلٍ تَتَزَيَّنُ امْرَأَتُهُ وَ تَخْرُجُ مِنْ بَابِ دَارِهَا فَهُوَ دَيُّوثٌ وَ لَا يَأْثَمُ مَنْ يُسَمِّيهِ دَيُّوثاً وَ الْمَرْأَةُ إِذَا خَرَجَتْ مِنْ بَابِ دَارِهَا مُتَزَيِّنَةً مُتَعَطِّرَةً وَ الزَّوْجُ بِذَلِكَ رَاضٍ يُبْنَى لِزَوْجِهَا بِكُلِّ قَدَمٍ بَيْتٌ فِي النَّارِ فَقَصِّرُوا أَجْنِحَةَ نِسَائِكُمْ وَ لَا تُطَوِّلُوهَا فَإِنَّ فِي تَقْصِيرِ أَجْنِحَتِهَا رِضًى وَ سُرُوراً وَ دُخُولَ الْجَنَّةِ بِغَيْرِ حِسَابٍ احْفَظُوا وَصِيَّتِي فِي أَمْرِ نِسَائِكُمْ حَتَّى تَنْجُوا مِنْ شِدَّةِ الْحِسَابِ وَ مَنْ لَمْ يَحْفَظْ وَصِيَّتِي فَمَا أَسْوَأَ حَالَهُ بَيْنَ يَدَيِ اللَّهِ وَ قَالَ ع النِّسَاءُ حَبَائِلُ الشَّيْطَان', 'Label': 1, 'text_length': 924, 'input_ids':

Our first intuition with the BERT tokeniser was not very fruitful.The tokeniser was not able to identify arabic words for exemple and was creatinng a unique token for all the sentence.We decided to try another tokenizer.

In [6]:
from transformers import XLMRobertaTokenizer
from datasets import load_dataset

# Charger le tokenizer XLM-RoBERTa
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Charger le dataset
dataset = load_dataset('csv', data_files={'train': 'train_cleaned_numeric_labels.csv'}, delimiter=',')

# Afficher le premier texte du dataset pour vérification
text = dataset['train']['Text'][0]
print("Texte original :")
print(text)

# Tokenisation manuelle du premier texte
tokens = tokenizer.tokenize(text)
print("\nTokens générés :")
print(tokens)

# Fonction de tokenisation pour le dataset
def tokenize_function(examples):
    return tokenizer(
        examples['Text'],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"  # Retourne des tenseurs PyTorch (optionnel)
    )

# Appliquer la tokenisation à l'ensemble du dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, batch_size=1000)

# Afficher les résultats de la tokenisation pour le premier exemple
print("\nIDs des tokens pour le premier exemple :")
print(tokenized_datasets['train'][0]['input_ids'])

print("\nMasque d'attention pour le premier exemple :")
print(tokenized_datasets['train'][0]['attention_mask'])

# Optionnel : Convertir les IDs en tokens pour vérification
input_ids = tokenized_datasets['train'][0]['input_ids']
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print("\nTokens reconstruits à partir des IDs :")
print(tokens)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

Texte original :
َ قَالَ النَّبِيُّ ص إِنِّي أَتَعَجَّبُ مِمَّنْ يَضْرِبُ امْرَأَتَهُ وَ هُوَ بِالضَّرْبِ أَوْلَى مِنْهَا لَا تَضْرِبُوا نِسَاءَكُمْ بِالْخَشَبِ فَإِنَّ فِيهِ الْقِصَاصَ وَ لَكِنِ اضْرِبُوهُنَّ بِالْجُوعِ وَ الْعُرْيِ حَتَّى تُرِيحُوا [تَرْبَحُوا] فِي الدُّنْيَا وَ الْآخِرَةِ وَ أَيُّمَا رَجُلٍ تَتَزَيَّنُ امْرَأَتُهُ وَ تَخْرُجُ مِنْ بَابِ دَارِهَا فَهُوَ دَيُّوثٌ وَ لَا يَأْثَمُ مَنْ يُسَمِّيهِ دَيُّوثاً وَ الْمَرْأَةُ إِذَا خَرَجَتْ مِنْ بَابِ دَارِهَا مُتَزَيِّنَةً مُتَعَطِّرَةً وَ الزَّوْجُ بِذَلِكَ رَاضٍ يُبْنَى لِزَوْجِهَا بِكُلِّ قَدَمٍ بَيْتٌ فِي النَّارِ فَقَصِّرُوا أَجْنِحَةَ نِسَائِكُمْ وَ لَا تُطَوِّلُوهَا فَإِنَّ فِي تَقْصِيرِ أَجْنِحَتِهَا رِضًى وَ سُرُوراً وَ دُخُولَ الْجَنَّةِ بِغَيْرِ حِسَابٍ احْفَظُوا وَصِيَّتِي فِي أَمْرِ نِسَائِكُمْ حَتَّى تَنْجُوا مِنْ شِدَّةِ الْحِسَابِ وَ مَنْ لَمْ يَحْفَظْ وَصِيَّتِي فَمَا أَسْوَأَ حَالَهُ بَيْنَ يَدَيِ اللَّهِ وَ قَالَ ع النِّسَاءُ حَبَائِلُ الشَّيْطَان

Tokens générés :
['▁', 'َ', '▁ق', 'َ', 'ال', 'َ', '▁الن',

Map:   0%|          | 0/190099 [00:00<?, ? examples/s]


IDs des tokens pour le premier exemple :
[0, 6, 278, 5081, 278, 1282, 278, 6608, 2897, 1325, 488, 179, 30877, 5668, 3212, 488, 323, 16102, 179, 1333, 278, 368, 278, 1195, 278, 2048, 2897, 1325, 577, 665, 488, 376, 2897, 323, 1060, 767, 278, 4995, 1060, 835, 488, 1325, 577, 8952, 1060, 835, 278, 4298, 278, 368, 278, 176, 577, 65, 278, 7947, 577, 431, 278, 676, 488, 1282, 4995, 2897, 835, 1060, 1325, 488, 1333, 278, 431, 1060, 593, 278, 902, 665, 488, 323, 1060, 176, 278, 258, 795, 278, 258, 746, 278, 4995, 1060, 835, 488, 1325, 577, 1778, 1706, 488, 926, 278, 2689, 278, 972, 577, 376, 1060, 676, 488, 1282, 1060, 3341, 278, 870, 278, 1325, 488, 2103, 278, 23132, 488, 323, 2897, 2103, 488, 4905, 488, 359, 1060, 1933, 488, 3092, 278, 29633, 278, 65, 278, 795, 278, 972, 488, 323, 488, 6, 29168, 1060, 835, 488, 1325, 577, 10482, 577, 323, 2897, 676, 488, 1282, 1060, 2048, 577, 15645, 488, 65, 278, 359, 1060, 1195, 577, 835, 1060, 179, 488, 6257, 278, 368, 2897, 902, 746, 577, 835, 488, 1402

This step was not mandatory, but the problem is with languages such as arabic,chinese or tukish where the characters are with a complex morphology and the words are not seperated, tokenizinng simplify the problem a lot

Another point was made here , we needed to prove which tokeniser gave us better results for most languages not only the ones we know . We decided to compare them using the codee below:

In [5]:
import numpy as np
import pandas as pd
import os, warnings
warnings.filterwarnings('ignore')


import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


from transformers import BertTokenizer, TFBertModel               #BERT
from transformers import DistilBertTokenizer, TFDistilBertModel    #DistilBERT
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel    #XLM-RoBERTa


from sklearn.model_selection import train_test_split


import gc

from tabulate import tabulate

os.environ["WANDB_API_KEY"] = "0"

In [6]:
def Init_TPU():

    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        REPLICAS = strategy.num_replicas_in_sync
        print("Connected to TPU Successfully:\n TPUs Initialised with Replicas:",REPLICAS)

        return strategy

    except ValueError:

        print("Connection to TPU Falied")
        print("Using default strategy for CPU and single GPU")
        strategy = tf.distribute.get_strategy()

        return strategy

strategy=Init_TPU()

Connection to TPU Falied
Using default strategy for CPU and single GPU


In [7]:

train_url = os.path.join('train_cleaned_numeric_labels.csv')
train_data = pd.read_csv(train_url, header='infer')
gc.collect()

4116

In [8]:
# Transformer Model Name
Bert_model = 'bert-base-multilingual-cased'
distilBert_model = 'distilbert-base-multilingual-cased'
xlmRoberta_model = 'jplu/tf-xlm-roberta-base'

# Define Tokenizer for each
Bert_toknzr = BertTokenizer.from_pretrained(Bert_model)
distilBert_toknzr = DistilBertTokenizer.from_pretrained(distilBert_model)
xlmRoberta_toknzr = XLMRobertaTokenizer.from_pretrained(xlmRoberta_model)


In [9]:
# Checking the output of tokenizer
sentence = 'Le temps est beau le ciel est bleu!'

print("BERT Model Tokenizer Output:",Bert_toknzr.convert_tokens_to_ids(list(Bert_toknzr.tokenize(sentence))))
print("DistilBERT Model Tokenizer Output:",distilBert_toknzr.convert_tokens_to_ids(list(distilBert_toknzr.tokenize(sentence))))
print("XLM-RoBERTa Model Tokenizer Output:",xlmRoberta_toknzr.convert_tokens_to_ids(list(xlmRoberta_toknzr.tokenize(sentence))))

BERT Model Tokenizer Output: [10281, 12358, 10176, 58817, 10141, 40229, 10176, 52816, 106]
DistilBERT Model Tokenizer Output: [10281, 12358, 10176, 58817, 10141, 40229, 10176, 52816, 106]
XLM-RoBERTa Model Tokenizer Output: [636, 4109, 437, 44551, 95, 6, 67708, 437, 123774, 38]


In [10]:
# Extraire uniquement la colonne 'Text' sous forme de liste simple de chaînes
train = train_data['Text'].values.tolist()

# Maximum length
max_len = 50

# Tokenisation avec BERT
train_encode_Bert = Bert_toknzr.batch_encode_plus(train, pad_to_max_length=True, max_length=max_len, truncation=True)

# Tokenisation avec DistilBERT
train_encode_DistilBert = distilBert_toknzr.batch_encode_plus(train, pad_to_max_length=True, max_length=max_len, truncation=True)

# Tokenisation avec XLM-RoBERTa
train_encode_XlmRoberta = xlmRoberta_toknzr.batch_encode_plus(train, pad_to_max_length=True, max_length=max_len, truncation=True)


In [11]:


test_size = 0.1

# BERT
x_tr_bert, x_val_bert, y_tr_bert, y_val_bert = train_test_split(train_encode_Bert['input_ids'], train_data.Label.values, test_size=test_size)

# DistilBERT
x_tr_Dbert, x_val_Dbert, y_tr_Dbert, y_val_Dbert = train_test_split(train_encode_DistilBert['input_ids'], train_data.Label.values, test_size=test_size)

# XLM-RoBERTa
x_tr_XR, x_val_XR, y_tr_XR, y_val_XR = train_test_split(train_encode_XlmRoberta['input_ids'], train_data.Label.values, test_size=test_size)

In [12]:
gc.collect()

0

In [13]:

AUTO = tf.data.experimental.AUTOTUNE
batch_size = 16 * strategy.num_replicas_in_sync

#BERT
tr_ds_bert = (tf.data.Dataset.from_tensor_slices((x_tr_bert, y_tr_bert)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))
val_ds_bert = (tf.data.Dataset.from_tensor_slices((x_val_bert, y_val_bert)).batch(batch_size).prefetch(AUTO))

#DistilBERT
tr_ds_Dbert = (tf.data.Dataset.from_tensor_slices((x_tr_Dbert, y_tr_Dbert)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))
val_ds_Dbert = (tf.data.Dataset.from_tensor_slices((x_val_Dbert, y_val_Dbert)).batch(batch_size).prefetch(AUTO))

#XLM-RoBERTa
tr_ds_XR = (tf.data.Dataset.from_tensor_slices((x_tr_XR, y_tr_XR)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))
val_ds_XR = (tf.data.Dataset.from_tensor_slices((x_val_XR, y_val_XR)).batch(batch_size).prefetch(AUTO))

In [14]:
gc.collect()


0

In [7]:
import tensorflow as tf
from transformers import TFBertModel, TFDistilBertModel, TFXLMRobertaModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

import os

os.environ['TF_USE_LEGACY_KERAS'] = '2'

# Définir les modèles pré-entraînés
Bert_model = 'bert-base-uncased'
distilBert_model = 'distilbert-base-uncased'
xlmRoberta_model = 'xlm-roberta-base'

max_len = 50  # Longueur maximale des séquences
def build_model():
    # Modèle BERT
    input_ids_bert = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask_bert = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    # Créer le modèle BERT
    bert = TFBertModel.from_pretrained(Bert_model)
    # Correct call method
    bert_outputs = bert(input_ids=input_ids_bert, attention_mask=attention_mask_bert)
    cls_token_bert = bert_outputs[0][:, 0, :]
    output_layer_bert = Dense(3, activation='softmax')(cls_token_bert)
    model_bert = Model(inputs=[input_ids_bert, attention_mask_bert], outputs=output_layer_bert)
    model_bert.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Modèle DistilBERT
    input_ids_dbert = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask_dbert = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    # Créer le modèle DistilBERT
    distilbert = TFDistilBertModel.from_pretrained(distilBert_model)
    # Correct call method
    distilbert_outputs = distilbert(input_ids=input_ids_dbert, attention_mask=attention_mask_dbert)
    cls_token_dbert = distilbert_outputs[0][:, 0, :]
    output_layer_dbert = Dense(3, activation='softmax')(cls_token_dbert)
    model_dbert = Model(inputs=[input_ids_dbert, attention_mask_dbert], outputs=output_layer_dbert)
    model_dbert.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Modèle XLM-RoBERTa
    input_ids_xlm = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask_xlm = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    # Créer le modèle XLM-RoBERTa
    xlm = TFXLMRobertaModel.from_pretrained(xlmRoberta_model)
    # Correct call method
    xlm_outputs = xlm(input_ids=input_ids_xlm, attention_mask=attention_mask_xlm)
    cls_token_xlm = xlm_outputs[0][:, 0, :]
    output_layer_xlm = Dense(3, activation='softmax')(cls_token_xlm)
    model_xlm = Model(inputs=[input_ids_xlm, attention_mask_xlm], outputs=output_layer_xlm)
    model_xlm.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model_bert, model_dbert, model_xlm

# Appel de la fonction
model_bert, model_Dbert, model_XLMRoberta = build_model()

# Vérifier les architectures des modèles
print("BERT Model Summary:")
model_bert.summary()

print("\nDistilBERT Model Summary:")
model_Dbert.summary()

print("\nXLM-RoBERTa Model Summary:")
model_XLMRoberta.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_5' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_5' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 50), dtype=int32, sparse=False, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 50), dtype=int32, sparse=False, name=attention_mask>
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

In [20]:
epochs = 30  # < change if you wish
n_steps = len(train_data) // batch_size
# Train BERT Model

model_bert.fit(tr_ds_bert,
          steps_per_epoch = n_steps,
          validation_data = val_ds_bert,
          epochs = epochs)

NameError: name 'model_bert' is not defined

In [21]:
model_Dbert.fit(tr_ds_Dbert,
          steps_per_epoch = n_steps,
          validation_data = val_ds_Dbert,
          epochs = epochs)

NameError: name 'model_Dbert' is not defined

In [22]:

model_XLMRoberta.fit(tr_ds_XR,
          steps_per_epoch = n_steps,
          validation_data = val_ds_XR,
          epochs = epochs)

NameError: name 'model_XLMRoberta' is not defined

In [23]:
# Evaluate BERT
res_bert = model_bert.evaluate(val_ds_bert, verbose=0)

# Evaluate DistilBERT
res_Dbert = model_Dbert.evaluate(val_ds_Dbert, verbose=0)

# Evaluate XLM-RoBERTa
res_XlmRoberta = model_XLMRoberta.evaluate(val_ds_XR, verbose=0)

#Tabulate Data
tab_data = [["BERT","30","Adam","128","1e-5",'{:.2%}'.format(res_bert[1])],
            ["DistilBERT","30","Adam","128","1e-5",'{:.2%}'.format(res_Dbert[1])],
            ["XLM-RoBERTa","30","Adam","128","1e-5",'{:.2%}'.format(res_XlmRoberta[1])]]

print(tabulate(tab_data, headers=['Models','Epochs','Optimizer','Batch Size','Learning Rate','Accuracy'], tablefmt='pretty'))

NameError: name 'model_bert' is not defined

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from transformers import BertTokenizer, TFBertModel
from transformers import DistilBertTokenizer, TFDistilBertModel
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel

from sklearn.model_selection import train_test_split
from tabulate import tabulate
import gc

# Suppression des avertissements
import warnings
warnings.filterwarnings('ignore')

# Configuration TPU si disponible
def Init_TPU():
    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        REPLICAS = strategy.num_replicas_in_sync
        print("Connected to TPU Successfully:\n TPUs Initialised with Replicas:", REPLICAS)
        return strategy
    except ValueError:
        print("Connection to TPU Failed")
        print("Using default strategy for CPU and single GPU")
        strategy = tf.distribute.get_strategy()
        return strategy

strategy = Init_TPU()

# Chargement des données
train_data = pd.read_csv('train_cleaned_numeric_labels.csv', header='infer')
gc.collect()

# Configuration des modèles
Bert_model = 'bert-base-multilingual-cased'
distilBert_model = 'distilbert-base-multilingual-cased'
xlmRoberta_model = 'jplu/tf-xlm-roberta-base'

# Initialisation des tokenizers
Bert_toknzr = BertTokenizer.from_pretrained(Bert_model)
distilBert_toknzr = DistilBertTokenizer.from_pretrained(distilBert_model)
xlmRoberta_toknzr = XLMRobertaTokenizer.from_pretrained(xlmRoberta_model)

# Préparation des données
train_texts = train_data['Text'].values.tolist()
max_len = 50

# Tokenisation
train_encode_Bert = Bert_toknzr.batch_encode_plus(
    train_texts,
    padding='max_length',
    max_length=max_len,
    truncation=True,
    return_attention_mask=True,
    return_token_type_ids=True
)

train_encode_DistilBert = distilBert_toknzr.batch_encode_plus(
    train_texts,
    padding='max_length',
    max_length=max_len,
    truncation=True,
    return_attention_mask=True
)

train_encode_XlmRoberta = xlmRoberta_toknzr.batch_encode_plus(
    train_texts,
    padding='max_length',
    max_length=max_len,
    truncation=True,
    return_attention_mask=True
)

# Division train/validation
test_size = 0.1

# BERT
X_tr_bert_ids = np.array(train_encode_Bert['input_ids'])
X_tr_bert_mask = np.array(train_encode_Bert['attention_mask'])
X_tr_bert_token_type = np.array(train_encode_Bert['token_type_ids'])

# DistilBERT
X_tr_dbert_ids = np.array(train_encode_DistilBert['input_ids'])
X_tr_dbert_mask = np.array(train_encode_DistilBert['attention_mask'])

# XLM-RoBERTa
X_tr_xlm_ids = np.array(train_encode_XlmRoberta['input_ids'])
X_tr_xlm_mask = np.array(train_encode_XlmRoberta['attention_mask'])

# Labels
y = train_data.Label.values

# BERT split
X_tr_bert_ids, X_val_bert_ids, y_tr, y_val = train_test_split(X_tr_bert_ids, y, test_size=test_size, random_state=42)
X_tr_bert_mask, X_val_bert_mask, _, _ = train_test_split(X_tr_bert_mask, y, test_size=test_size, random_state=42)
X_tr_bert_token_type, X_val_bert_token_type, _, _ = train_test_split(X_tr_bert_token_type, y, test_size=test_size, random_state=42)

# DistilBERT split
X_tr_dbert_ids, X_val_dbert_ids, _, _ = train_test_split(X_tr_dbert_ids, y, test_size=test_size, random_state=42)
X_tr_dbert_mask, X_val_dbert_mask, _, _ = train_test_split(X_tr_dbert_mask, y, test_size=test_size, random_state=42)

# XLM-RoBERTa split
X_tr_xlm_ids, X_val_xlm_ids, _, _ = train_test_split(X_tr_xlm_ids, y, test_size=test_size, random_state=42)
X_tr_xlm_mask, X_val_xlm_mask, _, _ = train_test_split(X_tr_xlm_mask, y, test_size=test_size, random_state=42)

# Configuration des datasets
AUTO = tf.data.experimental.AUTOTUNE
batch_size = 16 * strategy.num_replicas_in_sync

# BERT
tr_ds_bert = tf.data.Dataset.from_tensor_slices(((X_tr_bert_ids, X_tr_bert_mask, X_tr_bert_token_type), y_tr))
tr_ds_bert = tr_ds_bert.repeat().shuffle(2048).batch(batch_size).prefetch(AUTO)

val_ds_bert = tf.data.Dataset.from_tensor_slices(((X_val_bert_ids, X_val_bert_mask, X_val_bert_token_type), y_val))
val_ds_bert = val_ds_bert.batch(batch_size).prefetch(AUTO)

# DistilBERT
tr_ds_dbert = tf.data.Dataset.from_tensor_slices(((X_tr_dbert_ids, X_tr_dbert_mask), y_tr))
tr_ds_dbert = tr_ds_dbert.repeat().shuffle(2048).batch(batch_size).prefetch(AUTO)

val_ds_dbert = tf.data.Dataset.from_tensor_slices(((X_val_dbert_ids, X_val_dbert_mask), y_val))
val_ds_dbert = val_ds_dbert.batch(batch_size).prefetch(AUTO)

# XLM-RoBERTa
tr_ds_xlm = tf.data.Dataset.from_tensor_slices(((X_tr_xlm_ids, X_tr_xlm_mask), y_tr))
tr_ds_xlm = tr_ds_xlm.repeat().shuffle(2048).batch(batch_size).prefetch(AUTO)

val_ds_xlm = tf.data.Dataset.from_tensor_slices(((X_val_xlm_ids, X_val_xlm_mask), y_val))
val_ds_xlm = val_ds_xlm.batch(batch_size).prefetch(AUTO)

# Déterminer le nombre de classes
num_classes = len(np.unique(y))

# Construction des modèles
with strategy.scope():
    # BERT Model
    input_ids_bert = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask_bert = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    token_type_ids_bert = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")

    bert = TFBertModel.from_pretrained(Bert_model)
    bert_outputs = bert(input_ids=input_ids_bert,
                         attention_mask=attention_mask_bert,
                         token_type_ids=token_type_ids_bert)

    cls_token_bert = bert_outputs[0][:, 0, :]
    output_layer_bert = Dense(num_classes, activation='softmax')(cls_token_bert)

    model_bert = Model(inputs=[input_ids_bert, attention_mask_bert, token_type_ids_bert],
                       outputs=output_layer_bert)

    model_bert.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 tf.keras.metrics.F1Score(name='f1')]
    )

    # DistilBERT Model
    input_ids_dbert = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask_dbert = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    distilbert = TFDistilBertModel.from_pretrained(distilBert_model)
    distilbert_outputs = distilbert(input_ids=input_ids_dbert, attention_mask=attention_mask_dbert)

    cls_token_dbert = distilbert_outputs[0][:, 0, :]
    output_layer_dbert = Dense(num_classes, activation='softmax')(cls_token_dbert)

    model_dbert = Model(inputs=[input_ids_dbert, attention_mask_dbert], outputs=output_layer_dbert)

    model_dbert.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 tf.keras.metrics.F1Score(name='f1')]
    )

    # XLM-RoBERTa Model
    input_ids_xlm = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask_xlm = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    xlm = TFXLMRobertaModel.from_pretrained(xlmRoberta_model)
    xlm_outputs = xlm(input_ids=input_ids_xlm, attention_mask=attention_mask_xlm)

    cls_token_xlm = xlm_outputs[0][:, 0, :]
    output_layer_xlm = Dense(num_classes, activation='softmax')(cls_token_xlm)

    model_xlm = Model(inputs=[input_ids_xlm, attention_mask_xlm], outputs=output_layer_xlm)

    model_xlm.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 tf.keras.metrics.F1Score(name='f1')]
    )

# Entraînement des modèles (commenté pour l'instant)
epochs = 3
steps_per_epoch = len(X_tr_bert_ids) // batch_size

# Décommentez pour entraîner les modèles
"""
# BERT
history_bert = model_bert.fit(
    tr_ds_bert,
    validation_data=val_ds_bert,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    verbose=1
)

# DistilBERT
history_dbert = model_dbert.fit(
    tr_ds_dbert,
    validation_data=val_ds_dbert,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    verbose=1
)

# XLM-RoBERTa
history_xlm = model_xlm.fit(
    tr_ds_xlm,
    validation_data=val_ds_xlm,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    verbose=1
)
"""

# Évaluation des modèles
print("Évaluation des modèles...")

res_bert = model_bert.evaluate(val_ds_bert, verbose=1)
res_dbert = model_dbert.evaluate(val_ds_dbert, verbose=1)
res_xlm = model_xlm.evaluate(val_ds_xlm, verbose=1)

# Création du tableau de métriques
metrics_names = ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']

tab_data = [
    ["BERT", epochs, "Adam", batch_size, "1e-5"] + [f"{metric:.4f}" for metric in res_bert],
    ["DistilBERT", epochs, "Adam", batch_size, "1e-5"] + [f"{metric:.4f}" for metric in res_dbert],
    ["XLM-RoBERTa", epochs, "Adam", batch_size, "1e-5"] + [f"{metric:.4f}" for metric in res_xlm]
]

headers = ['Model', 'Epochs', 'Optimizer', 'Batch Size', 'Learning Rate'] + metrics_names
print(tabulate(tab_data, headers=headers, tablefmt='pretty'))

# Prédictions sur les données de validation
bert_preds = np.argmax(model_bert.predict(val_ds_bert), axis=1)
dbert_preds = np.argmax(model_dbert.predict(val_ds_dbert), axis=1)
xlm_preds = np.argmax(model_xlm.predict(val_ds_xlm), axis=1)

# Calcul de métriques par classe pour chaque modèle
from sklearn.metrics import classification_report

print("\nRapport de classification BERT:")
print(classification_report(y_val, bert_preds))

print("\nRapport de classification DistilBERT:")
print(classification_report(y_val, dbert_preds))

print("\nRapport de classification XLM-RoBERTa:")
print(classification_report(y_val, xlm_preds))

# Calcul du temps d'inférence moyen pour chaque modèle
import time

# Fonction pour mesurer le temps d'inférence
def measure_inference_time(model, dataset, num_runs=10):
    times = []
    for _ in range(num_runs):
        start_time = time.time()
        model.predict(dataset, verbose=0)
        end_time = time.time()
        times.append(end_time - start_time)
    return np.mean(times)

print("\nMesure du temps d'inférence...")
bert_time = measure_inference_time(model_bert, val_ds_bert)
dbert_time = measure_inference_time(model_dbert, val_ds_dbert)
xlm_time = measure_inference_time(model_xlm, val_ds_xlm)

time_data = [
    ["BERT", f"{bert_time:.4f}"],
    ["DistilBERT", f"{dbert_time:.4f}"],
    ["XLM-RoBERTa", f"{xlm_time:.4f}"]
]

print(tabulate(time_data, headers=['Model', 'Avg. Inference Time (s)'], tablefmt='pretty'))

# Matrice de confusion pour chaque modèle
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(num_classes), yticklabels=range(num_classes))
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{model_name}.png')
    plt.close()

plot_confusion_matrix(y_val, bert_preds, 'BERT')
plot_confusion_matrix(y_val, dbert_preds, 'DistilBERT')
plot_confusion_matrix(y_val, xlm_preds, 'XLM-RoBERTa')

print("\nMatrices de confusion enregistrées")

# Calcul de la taille du modèle et du nombre de paramètres
bert_size = model_bert.count_params() / 1_000_000
dbert_size = model_dbert.count_params() / 1_000_000
xlm_size = model_xlm.count_params() / 1_000_000

size_data = [
    ["BERT", f"{bert_size:.2f}M"],
    ["DistilBERT", f"{dbert_size:.2f}M"],
    ["XLM-RoBERTa", f"{xlm_size:.2f}M"]
]

print(tabulate(size_data, headers=['Model', 'Parameters (M)'], tablefmt='pretty'))

Connection to TPU Failed
Using default strategy for CPU and single GPU


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_6' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for attention_mask.

Call arguments received by layer 'tf_bert_model_6' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 50), dtype=int32, sparse=False, name=input_ids>
  • attention_mask=<KerasTensor shape=(None, 50), dtype=int32, sparse=False, name=attention_mask>
  • token_type_ids=<KerasTensor shape=(None, 50), dtype=int32, sparse=False, name=token_type_ids>
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_st